### Imports

In [1]:
import tensorflow as tf 
import tensorflow_io as tfio
import tensorflow_probability as tfp

print(tf.__version__)
print('Listing all GPU resources:')
print(tf.config.experimental.list_physical_devices('GPU'))
print()
import tensorflow.keras as keras
print(tfp.__version__)
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
import pickle
import os

2.2.0
Listing all GPU resources:
[]

0.9.0


## Parameters

In [2]:
BATCH_SIZE = 128
FILTERS = 32
EPOCHS = 100
DATA_PATH = '../data/Task01_BrainTumour.h5'
PICKLE_PATH = 'Layer_1_hist.pkl'

DATA_SIZE = 60000

PRIOR_MU = 0
PRIOR_SIGMA = 10

### Data

### Functions

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
    
    class KLDivergence:
        def __init__(self, q_dist, p_dist):
            self.q_dist = q_dist         
            self.p_dist = p_dist
        def call(self):
            return tfp.distributions.kl_divergence(self.q_dist, self.p_dist)

    def mean_binary_crossentropy(y, y_pred):
        return tf.reduce_mean(keras.losses.binary_crossentropy(y, y_pred))

    def sum_binary_crossentropy(y, y_pred):
        return DATA_SIZE * mean_binary_crossentropy(y, y_pred)

    def likelihood_loss(y, y_pred):
        return sum_binary_crossentropy(y, y_pred)
    
    posterior_fn = tfp.layers.default_mean_field_normal_fn(
                  loc_initializer=tf.random_normal_initializer(
                      mean=PRIOR_MU, stddev=0.05),
                  untransformed_scale_initializer=tf.random_normal_initializer(
                      mean=np.log(np.exp(PRIOR_SIGMA) - 1), stddev=0.05))

    prior_fn = tfp.layers.default_mean_field_normal_fn(
                      loc_initializer=tf.random_normal_initializer(
                          mean=PRIOR_MU, stddev=0.0),
                      untransformed_scale_initializer=tf.random_normal_initializer(
                          mean=np.log(np.exp(PRIOR_SIGMA) - 1), stddev=0))

    flipout_params = dict(kernel_size=(3, 3), activation="relu", padding="same", 
                  kernel_prior_fn=prior_fn,
                  bias_prior_fn=prior_fn,
                  kernel_posterior_fn=posterior_fn,
                  bias_posterior_fn=posterior_fn,
                  kernel_divergence_fn=None,
                  bias_divergence_fn=None)

    params_final = dict(kernel_size=(1, 1), activation="sigmoid", padding="same", 
                      data_format="channels_last",
                  kernel_initializer="he_uniform")

    params = dict(kernel_size=(3, 3), activation="relu",
                  padding="same", data_format="channels_last",
                  kernel_initializer="he_uniform")
    
    Xy_train = tf.data.Dataset.zip((tfio.IODataset.from_hdf5(DATA_PATH, '/imgs_train', dtype=tf.float32), 
                                tfio.IODataset.from_hdf5(DATA_PATH, '/msks_train', dtype=tf.float32))
                              ).batch(BATCH_SIZE).prefetch(2)
    Xy_test = tf.data.Dataset.zip((tfio.IODataset.from_hdf5(DATA_PATH, '/imgs_testing', dtype=tf.float32), 
                               tfio.IODataset.from_hdf5(DATA_PATH, '/msks_testing', dtype=tf.float32))
                             ).batch(BATCH_SIZE).prefetch(2)

    input_layer = keras.layers.Input(shape=(144, 144, 4), name="input_layer")

    encoder_1_a = tfp.layers.Convolution2DFlipout(FILTERS, name='encoder_1_a', **flipout_params)(input_layer)
    encoder_1_b = keras.layers.Conv2D(FILTERS, name='encoder_1_b', **params)(encoder_1_a)
    downsample_1 = keras.layers.MaxPool2D(name='downsample_1')(encoder_1_b)

    encoder_2_a = keras.layers.Conv2D(FILTERS*2, name='encoder_2_a', **params)(downsample_1)
    encoder_2_b = keras.layers.Conv2D(FILTERS*2, name='encoder_2_b', **params)(encoder_2_a)
    downsample_2 = keras.layers.MaxPool2D(name='downsample_2')(encoder_2_b)

    encoder_3_a = keras.layers.Conv2D(FILTERS*4, name='encoder_3_a', **params)(downsample_2)
    encoder_3_b = keras.layers.Conv2D(FILTERS*4, name='encoder_3_b', **params)(encoder_3_a)
    downsample_3 = keras.layers.MaxPool2D(name='downsample_3')(encoder_3_b)

    encoder_4_a = keras.layers.Conv2D(FILTERS*8, name='encoder_4_a', **params)(downsample_3)
    encoder_4_b = keras.layers.Conv2D(FILTERS*8, name='encoder_4_b', **params)(encoder_4_a)
    downsample_4 = keras.layers.MaxPool2D(name='downsample_4')(encoder_4_b)


    encoder_5_a = keras.layers.Conv2D(FILTERS*16, name='encoder_5_a', **params)(downsample_4)
    encoder_5_b = keras.layers.Conv2D(FILTERS*16, name='encoder_5_b', **params)(encoder_5_a)


    upsample_4 = keras.layers.UpSampling2D(name='upsample_4', size=(2, 2), interpolation="bilinear")(encoder_5_b)
    concat_4 = keras.layers.concatenate([upsample_4, encoder_4_b], name='concat_4')
    decoder_4_a = keras.layers.Conv2D(FILTERS*8, name='decoder_4_a', **params)(concat_4)
    decoder_4_b = keras.layers.Conv2D(FILTERS*8, name='decoder_4_b', **params)(decoder_4_a)


    upsample_3 = keras.layers.UpSampling2D(name='upsample_3', size=(2, 2), interpolation="bilinear")(decoder_4_b)
    concat_3 = keras.layers.concatenate([upsample_3, encoder_3_b], name='concat_3')
    decoder_3_a = keras.layers.Conv2D(FILTERS*4, name='decoder_3_a', **params)(concat_3)
    decoder_3_b = keras.layers.Conv2D(FILTERS*4, name='decoder_3_b', **params)(decoder_3_a)


    upsample_2 = keras.layers.UpSampling2D(name='upsample_2', size=(2, 2), interpolation="bilinear")(decoder_3_b)
    concat_2 = keras.layers.concatenate([upsample_2, encoder_2_b], name='concat_2')
    decoder_2_a = keras.layers.Conv2D(FILTERS*2, name='decoder_2_a', **params)(concat_2)
    decoder_2_b = keras.layers.Conv2D(FILTERS*2, name='decoder_2_b', **params)(decoder_2_a)


    upsample_1 = keras.layers.UpSampling2D(name='upsample_1', size=(2, 2), interpolation="bilinear")(decoder_2_b)
    concat_1 = keras.layers.concatenate([upsample_1, encoder_1_b], name='concat_1')
    decoder_1_a = keras.layers.Conv2D(FILTERS, name='decoder_1_a', **params)(concat_1)
    decoder_1_b = keras.layers.Conv2D(FILTERS, name='decoder_1_b', **params)(decoder_1_a)

    output_layer = keras.layers.Conv2D(name="output_layer",
                                    filters=1, **params_final)(decoder_1_b)

    print()
    print('Input size:', input_layer.shape)
    print('Output size:', output_layer.shape)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    for layer in model.layers:
        if type(layer) == tfp.python.layers.conv_variational.Conv2DFlipout:
            layer.add_loss(KLDivergence(layer.kernel_posterior, layer.kernel_prior).call)
            layer.add_loss(KLDivergence(layer.bias_posterior, layer.bias_prior).call)
    
    model.compile(optimizer=keras.optimizers.Nadam(learning_rate=1e-4), 
                  loss=likelihood_loss, 
                  metrics=[likelihood_loss, mean_binary_crossentropy])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [ ]:
bias_prior = model.layers[1].bias_prior
bias_posterior = model.layers[1].bias_posterior
kernel_prior = model.layers[1].kernel_prior
kernel_posterior = model.layers[1].kernel_posterior

In [ ]:
print('KL Divergence at Initialization:', '%0.3f' % (tfp.distributions.kl_divergence(bias_posterior, bias_prior) \
      + tfp.distributions.kl_divergence(kernel_posterior, kernel_prior)).numpy())

In [ ]:
print(model.losses)

### Train

In [ ]:
print("-" * 30)
print("Fitting model with training data ...")
print("-" * 30)

print("Step 3, training the model started at {}".format(datetime.datetime.now()))
start_time = time.time()

history = model.fit(Xy_train,
          validation_data=Xy_test,
          epochs=EPOCHS, verbose=1)

print("Total time elapsed for training = {} seconds".format(time.time() - start_time))
print("Training finished at {}".format(datetime.datetime.now()))

### Save model

In [ ]:
# Save the model
# serialize weights to HDF5 and tensorflow_model format
model.save_weights("layer_1_bayesian.h5")
print("Saved model to disk (.h5)")
model.save_weights('layer_1_bayesian')
print("Saved model to disk (.tf)")

#Save history
history_dict = history.history
with open(PICKLE_PATH, 'wb') as file_pi:
        pickle.dump(history_dict, file_pi)
print('Pickled training history')